In [201]:
import pandas as pd
import math
import numpy as np
import pygal
%matplotlib inline
pd.set_option('display.max_rows', None)
from IPython.display import display, HTML
base_html = """
<!DOCTYPE html>
<html>
  <head>
  <script type="text/javascript" src="http://kozea.github.com/pygal.js/javascripts/svg.jquery.js"></script>
  <script type="text/javascript" src="https://kozea.github.io/pygal.js/2.0.x/pygal-tooltips.min.js""></script>
  </head>
  <body>
    <figure>
      {rendered_chart}
    </figure>
  </body>
</html>
"""

In [202]:
df=pd.read_csv("./csv/OnlineNewsPopularity.csv")

In [203]:
df.isna().sum() 

url                               0
 timedelta                        0
 n_tokens_title                   0
 n_tokens_content                 0
 n_unique_tokens                  0
 n_non_stop_words                 0
 n_non_stop_unique_tokens         0
 num_hrefs                        0
 num_self_hrefs                   0
 num_imgs                         0
 num_videos                       0
 average_token_length             0
 num_keywords                     0
 data_channel_is_lifestyle        0
 data_channel_is_entertainment    0
 data_channel_is_bus              0
 data_channel_is_socmed           0
 data_channel_is_tech             0
 data_channel_is_world            0
 kw_min_min                       0
 kw_max_min                       0
 kw_avg_min                       0
 kw_min_max                       0
 kw_max_max                       0
 kw_avg_max                       0
 kw_min_avg                       0
 kw_max_avg                       0
 kw_avg_avg                 

In [204]:
ls_cont = ['n_tokens_title', 'n_tokens_content', 'n_unique_tokens', 'n_non_stop_words', 
           'n_non_stop_unique_tokens', 'num_hrefs', 'num_self_hrefs', 'num_imgs', 'num_videos', 
           'average_token_length', 'num_keywords', 'kw_min_min', 'kw_max_min', 'kw_avg_min', 
           'kw_min_max', 'kw_max_max', 'kw_avg_max', 'kw_min_avg', 'kw_max_avg', 'kw_avg_avg', 
           'self_reference_min_shares', 'self_reference_max_shares', 'self_reference_avg_sharess', 
           'LDA_00', 'LDA_01', 'LDA_02', 'LDA_03', 'LDA_04', 'global_subjectivity', 
           'global_sentiment_polarity', 'global_rate_positive_words', 'global_rate_negative_words',
           'rate_positive_words', 'rate_negative_words', 'avg_positive_polarity', 'min_positive_polarity',
           'max_positive_polarity', 'avg_negative_polarity', 'min_negative_polarity', 
           'max_negative_polarity', 'title_subjectivity', 'title_sentiment_polarity', 
           'abs_title_subjectivity', 'abs_title_sentiment_polarity']
ls_disc = ['data_channel_is_lifestyle', 'data_channel_is_entertainment', 'data_channel_is_bus', 
           'data_channel_is_socmed', 'data_channel_is_tech', 'data_channel_is_world', 'weekday_is_monday',
           'weekday_is_tuesday', 'weekday_is_wednesday', 'weekday_is_thursday', 'weekday_is_friday',
           'weekday_is_saturday', 'weekday_is_sunday', 'is_weekend']

In [205]:
df.columns = [x.strip() for x in df.columns]

In [206]:
def plot_hist(df,value,route=""):
    info=df[value].describe()
    r=info["max"]-info["min"]
    k=math.ceil(1+3.322*math.log10(info["count"]))
    a=int(round(r/k))
    ranges=[]
    for i in range(int(info["min"]),int(info["max"]+2),a):
        ranges.append(i)
    aux=np.histogram(df[value], bins=ranges)
    histograma=[]
    for i in range(len(aux[0])):
        histograma.append([aux[0][i],aux[1][i],aux[1][i+1]])
    hist = pygal.Histogram()
    hist.add(value,histograma)
    if route!='':
        return hist.render_to_png(route)
    display(HTML(base_html.format(rendered_chart=hist.render(is_unicode=True))))

In [207]:
plot_hist(df,"shares")

In [208]:
df["shares"].describe(percentiles = [x/10 for x in range(10)] + [0.01, 0.05, 0.95, 0.99])

count     39644.000000
mean       3395.380184
std       11626.950749
min           1.000000
0%            1.000000
1%          381.000000
5%          584.000000
10%         708.000000
20%         870.600000
30%        1000.000000
40%        1200.000000
50%        1400.000000
60%        1800.000000
70%        2300.000000
80%        3400.000000
90%        6200.000000
95%       10800.000000
99%       31657.000000
max      843300.000000
Name: shares, dtype: float64

In [209]:
aux=df[(df["shares"]>381) & (df["shares"]<10800)]

In [210]:
plot_hist(aux,"shares")

In [211]:
df.shape[0]

39644

In [212]:
aux.shape

(37246, 61)

In [213]:
aux.shape[0]/df.shape[0]

0.939511653718091

In [214]:
aux["shares"].describe(percentiles = [x/10 for x in range(10)] + [0.01, 0.05, 0.95, 0.99])

count    37246.000000
mean      2089.679643
std       1861.995344
min        382.000000
0%         382.000000
1%         461.000000
5%         605.000000
10%        715.000000
20%        867.000000
30%       1000.000000
40%       1200.000000
50%       1400.000000
60%       1700.000000
70%       2100.000000
80%       2900.000000
90%       4500.000000
95%       6400.000000
99%       9400.000000
max      10700.000000
Name: shares, dtype: float64

In [215]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression

In [216]:
lr = LinearRegression()

In [217]:
lr.fit(X = aux[ls_cont+ls_disc], y = aux["shares"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [218]:
lr.intercept_

232894.68123545963

In [219]:
lr.score(X = aux[ls_cont+ls_disc], y = aux["shares"])

0.08645011424735537

In [220]:
lr.coef_

array([-2.71807796e+00,  8.72816795e-02, -4.69345078e+01, -9.82035152e+01,
       -1.41446399e+02,  7.33225741e+00, -1.79920316e+01,  3.74243064e+00,
        2.99519835e+00, -1.47651479e+02,  3.20409607e+01,  6.46877141e-01,
        2.94785788e-02, -2.44812820e-01, -2.44722082e-04, -1.85588391e-04,
       -7.77810295e-04, -5.68687221e-02, -7.19191073e-02,  5.68101377e-01,
        6.90740837e-05, -1.53199448e-04,  2.89745513e-03, -2.31343651e+05,
       -2.32045168e+05, -2.32163600e+05, -2.32118740e+05, -2.31862880e+05,
        4.55977618e+02, -3.61589281e+02,  4.03884683e+02, -8.15190884e+02,
        6.66091267e+02,  5.14187943e+02, -1.29607022e+01, -4.30751325e+02,
       -6.05584069e+01, -9.66060172e+01, -1.64635659e+01,  7.83398864e+01,
        1.33361540e+02,  1.20482292e+02,  2.54084515e+02,  1.00116555e+01,
       -1.95587170e+02, -3.23173913e+02, -3.10791797e+02,  3.39253848e+02,
        2.29404206e+02, -9.33981599e+01,  6.03774474e+00, -9.09676863e+01,
       -8.10219783e+01, -

In [221]:
lr.intercept_

232894.68123545963

In [20]:
from sklearn.linear_model import LogisticRegression

In [21]:
lr = LogisticRegression(n_jobs=-1)

In [63]:
aux["tgt"] = (aux["shares"] > 2089.679643 ).astype(int)

/home/arturo/env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
aux.head()

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares,tgt
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593,0
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711,0
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500,0
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200,0
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505,0


In [24]:
lr = lr.fit(X=aux[ls_cont+ls_disc], y = aux["tgt"])

/home/arturo/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/arturo/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


In [25]:
lr.score(X=aux[ls_cont+ls_disc], y=aux["tgt"])

0.6829189711646888

# Credit Scoring

In [168]:
aux["tgt"].value_counts(True)

0    0.688128
1    0.311872
Name: tgt, dtype: float64

In [169]:
for var in ls_cont:
    aux[f"C_{var}"] = pd.cut(aux[var], bins=8).astype(str)

/home/arturo/env/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [170]:
def IV(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux["IV"] = (aux["%no_evento"] - aux["%evento"])*aux["WOE"]
    return aux["IV"].sum()

In [171]:
new_disc = [x for x in aux.columns if x.startswith("C_")] + ls_disc

In [94]:
df_iv = pd.DataFrame(columns=["iv"])
for var in new_disc:
    df_iv.loc[var, "iv"] = IV(df = aux, var = var, tgt = "tgt")

/home/arturo/env/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [117]:
total=pd.DataFrame()
for i in range(2,15):
    for var in ls_cont:
        aux[f"C_{var}"] = pd.cut(aux[var], bins=i).astype(str)
    new_disc = [x for x in aux.columns if x.startswith("C_")] + ls_disc
    df_iv = pd.DataFrame(columns=["iv"])
    for var in new_disc:
        df_iv.loc[var, "iv"] = IV(df = aux, var = var, tgt = "tgt")
    total["{}".format(i)]=df_iv["iv"]
    print(i)

/home/arturo/env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/arturo/env/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


2
3
4
5
6
7
8
9
10
11
12
13
14


In [119]:
total

,2,3,4,5,6,7,8,9,10,11,12,13,14
C_timedelta,0.00270985,0.00989059,0.016443,0.0203362,0.0212177,0.020903,0.0198519,0.0209723,0.024114,0.024113,0.0269076,0.0247709,0.0255737
C_n_tokens_title,0.00323669,0.00777265,0.00719354,0.00591093,0.00933421,inf,inf,inf,inf,inf,inf,inf,inf
C_n_tokens_content,0.00276951,0.00426534,0.00588078,0.0085168,0.010685,0.012854,0.0135814,0.015183,0.0145038,inf,inf,0.0138493,inf
C_n_unique_tokens,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_n_non_stop_words,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_n_non_stop_unique_tokens,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_num_hrefs,0.000593196,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_num_self_hrefs,0.000828945,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_num_imgs,0.0012102,0.00198266,0.00182432,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
C_num_videos,6.7921e-05,0.001872,0.000907339,0.000367917,0.00287952,0.00486977,0.00316583,0.00384443,0.00474865,inf,0.00428623,inf,inf


In [134]:
total.T

,C_timedelta,C_n_tokens_title,C_n_tokens_content,C_n_unique_tokens,C_n_non_stop_words,C_n_non_stop_unique_tokens,C_num_hrefs,C_num_self_hrefs,C_num_imgs,C_num_videos,...,data_channel_is_tech,data_channel_is_world,weekday_is_monday,weekday_is_tuesday,weekday_is_wednesday,weekday_is_thursday,weekday_is_friday,weekday_is_saturday,weekday_is_sunday,is_weekend
2,0.00270985,0.00323669,0.00276951,inf,inf,inf,0.000593196,0.000828945,0.0012102,6.7921e-05,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
3,0.00989059,0.00777265,0.00426534,inf,inf,inf,inf,inf,0.00198266,0.001872,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
4,0.016443,0.00719354,0.00588078,inf,inf,inf,inf,inf,0.00182432,0.000907339,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
5,0.0203362,0.00591093,0.0085168,inf,inf,inf,inf,inf,inf,0.000367917,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
6,0.0212177,0.00933421,0.010685,inf,inf,inf,inf,inf,inf,0.00287952,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
7,0.020903,inf,0.012854,inf,inf,inf,inf,inf,inf,0.00486977,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
8,0.0198519,inf,0.0135814,inf,inf,inf,inf,inf,inf,0.00316583,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
9,0.0209723,inf,0.015183,inf,inf,inf,inf,inf,inf,0.00384443,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
10,0.024114,inf,0.0145038,inf,inf,inf,inf,inf,inf,0.00474865,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353
11,0.024113,inf,inf,inf,inf,inf,inf,inf,inf,inf,...,0.0258721,0.0822831,0.00151351,0.00447295,0.00446024,0.00192198,0.000178396,0.0291338,0.0184555,0.0505353


In [138]:
tot=total.T

In [156]:
for j in tot.columns:
    tmp=tot[j][tot[j].index[0]]
    index=tot[j].index[0]
    for i in tot[j].index:
        if tot[j][i]:
            break
        if int(tot[j][i]) > tmp:
            tmp=int(tot[j][i])
            index=i
    print(j,index)

C_timedelta 2
C_n_tokens_title 2
C_n_tokens_content 2
C_n_unique_tokens 2
C_n_non_stop_words 2
C_n_non_stop_unique_tokens 2
C_num_hrefs 2
C_num_self_hrefs 2
C_num_imgs 2
C_num_videos 2
C_average_token_length 2
C_num_keywords 2
C_kw_min_min 2
C_kw_max_min 2
C_kw_avg_min 2
C_kw_min_max 2
C_kw_max_max 2
C_kw_avg_max 2
C_kw_min_avg 2
C_kw_max_avg 2
C_kw_avg_avg 2
C_self_reference_min_shares 2
C_self_reference_max_shares 2
C_self_reference_avg_sharess 2
C_LDA_00 2
C_LDA_01 2
C_LDA_02 2
C_LDA_03 2
C_LDA_04 2
C_global_subjectivity 2
C_global_sentiment_polarity 2
C_global_rate_positive_words 2
C_global_rate_negative_words 2
C_rate_positive_words 2
C_rate_negative_words 2
C_avg_positive_polarity 2
C_min_positive_polarity 2
C_max_positive_polarity 2
C_avg_negative_polarity 2
C_min_negative_polarity 2
C_max_negative_polarity 2
C_title_subjectivity 2
C_title_sentiment_polarity 2
C_abs_title_subjectivity 2
C_abs_title_sentiment_polarity 2
data_channel_is_lifestyle 2
data_channel_is_entertainment 2


In [164]:
tot=tot.replace([np.inf, -np.inf], np.nan).dropna(axis=1)

In [175]:
final=tot.idxmax()

In [178]:
final=final.drop(["C_timedelta"])

In [179]:
final.to_json("bins.json")

In [95]:
df_iv.sort_values(by = "iv", ascending=False)

,iv
C_kw_avg_avg,inf
C_num_imgs,inf
C_n_tokens_title,inf
C_kw_avg_min,inf
C_kw_max_min,inf
C_kw_min_min,inf
C_global_rate_negative_words,inf
C_self_reference_min_shares,inf
C_kw_max_avg,inf
C_num_self_hrefs,inf


In [31]:
def WOE(df, var, tgt):
    aux = df[[var, tgt]].groupby(var).agg(["count", "sum"])
    aux["evento"] = aux[tgt, "sum"]
    aux["no_evento"] = aux[tgt, "count"] - aux[tgt, "sum"]
    aux["%evento"] = aux["evento"] / aux["evento"].sum()
    aux["%no_evento"] = aux["no_evento"] / aux["no_evento"].sum()
    aux["WOE"] = np.log(aux["%no_evento"] / aux["%evento"])
    aux.columns = aux.columns.droplevel(1)
    aux = aux[["WOE"]].reset_index().rename(columns={"WOE": f"W_{var}"})
    df = df.merge(aux, on = var, how = "left")
    return df

In [35]:
for col in ls_disc:
    aux = WOE(df = aux, var = col, tgt = "shares")

,data_channel_is_lifestyle,W_data_channel_is_lifestyle
0,0.0,-0.000003
1,1.0,0.000047


,data_channel_is_entertainment,W_data_channel_is_entertainment
0,0.0,0.000015
1,1.0,-0.000086


,data_channel_is_bus,W_data_channel_is_bus
0,0.0,0.000004
1,1.0,-0.000021


,data_channel_is_socmed,W_data_channel_is_socmed
0,0.0,-0.000009
1,1.0,0.000110


,data_channel_is_tech,W_data_channel_is_tech
0,0.0,-0.000013
1,1.0,0.000048


...

,weekday_is_thursday,W_weekday_is_thursday
0,0.0,0.000005
1,1.0,-0.000022


,weekday_is_friday,W_weekday_is_friday
0,0.0,4.044219e-07
1,1.0,-2.423782e-06


,weekday_is_saturday,W_weekday_is_saturday
0,0.0,-0.000007
1,1.0,0.000091


,weekday_is_sunday,W_weekday_is_sunday
0,0.0,-0.000008
1,1.0,0.000087


,is_weekend,W_is_weekend
0,0.0,-0.000017
1,1.0,0.000089


In [36]:
ls_woe = [x for x in df.columns if x.startswith("W")]

In [37]:
lr = LogisticRegression(n_jobs=-1)

In [38]:
lr.fit(X=aux[ls_woe], y=df["shares"])

/home/arturo/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/arturo/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/arturo/env/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:1544: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=-1, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [39]:
lr.predict(df[ls_woe]).sum()

40970600

In [40]:
lr.score(X=df[ls_woe], y=df["shares"])

0.06196638565215057

In [43]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_true=aux["shares"], y_score=lr.predict(X=aux[ls_woe]))

ValueError: multiclass format is not supported

In [180]:
data1 = """
class    precision   recall 
<18      0.0125         12           
18-24    0.0250         16 
25-34    0.00350         4
"""
data2 = """
class    precision   recall 
<18      0               0           
18-24    0.25            6 
25-34    0.35            5
"""

#create 2 df for sample
df1 = pd.read_csv(pd.compat.StringIO(data1), sep='\s+')
df1.name = "Dataframe1"
df2 = pd.read_csv(pd.compat.StringIO(data2), sep='\s+')
df2.name = "Dataframe2"
print(df1);print(df2)

writer = pd.ExcelWriter('e:\\test.xlsx',engine='xlsxwriter')
workbook=writer.book
worksheet=workbook.add_worksheet('Result')
writer.sheets['Result'] = worksheet
worksheet.write_string(0, 0, df1.name)

df1.to_excel(writer,sheet_name='Result',startrow=1 , startcol=0)
worksheet.write_string(df1.shape[0] + 4, 0, df2.name)
df2.to_excel(writer,sheet_name='Result',startrow=df1.shape[0] + 5, startcol=0)

AttributeError: module 'pandas.compat' has no attribute 'StringIO'

In [193]:
from openpyxl.writer.Excel import ExcelWriter
writer = ExcelWriter('output.xlsx')
df1.to_Excel(writer, 'sheet1')
df2.to_Excel(writer, 'sheet2')
writer.save()

ModuleNotFoundError: No module named 'openpyxl.writer.Excel'

In [185]:
!pip install XlsxWriter

     |████████████████████████████████| 143kB 642kB/s eta 0:00:01
Could not install packages due to an EnvironmentError: [Errno 13] Permission denied: '/usr/lib/python3.7/site-packages/XlsxWriter-1.2.1.dist-info'
Consider using the `--user` option or check the permissions.



In [191]:
!python -m pip install --user openpyxl 

  Using cached https://files.pythonhosted.org/packages/6f/af/88ff9eef0b8f665aee1111ac6cede5ad12190c5bd726242bd2b26fc21b32/openpyxl-3.0.0.tar.gz
  Using cached https://files.pythonhosted.org/packages/f0/da/572cbc0bc582390480bbd7c4e93d14dc46079778ed915b505dc494b37c57/jdcal-1.4.1-py2.py3-none-any.whl
  Running setup.py install for openpyxl ... done


In [194]:
from openpyxl import Workbook

In [195]:
from openpyxl.writer.Excel import ExcelWriter

ModuleNotFoundError: No module named 'openpyxl.writer.Excel'

In [196]:
from openpyxl.utils.dataframe import dataframe_to_rows
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(df, index=True, header=True):
    ws.append(r)

In [198]:
wb = Workbook()
ws = wb.active

for r in dataframe_to_rows(tot, index=True, header=True):
    ws.append(r)

for cell in ws['A'] + ws[1]:
    cell.style = 'Pandas'

wb.save("pandas_openpyxl.xlsx")